In [85]:
import pandas as pd
from textblob import TextBlob
import string    


tab1 = "./hair_dryer.tsv"

df = pd.read_csv(tab1, sep='\t', header=0)

df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,34678741,R9T1FE2ZX2X04,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,Beauty,5,0,0,N,Y,Works great,Works great!,8/31/2015
1,US,11599505,RE36JAD5V53PO,B0009XH6V4,670161917,andis micro turbo hair dryer,Beauty,4,0,0,N,Y,I love travel blow dryers because they are eas...,"This dries my hair faster that bigger, more po...",8/31/2015
2,US,2282190,RIDHM8B7SCCV3,B0007NZPY6,16483457,conair pro hair dryer,Beauty,5,0,1,N,Y,Five Stars,Love this dryer!,8/31/2015
3,US,43669858,R14QGWPCHU9LSE,B00BB8ZIW0,253917972,remington silk ceramic professional hair dryer,Beauty,5,0,0,N,Y,Five Stars,styling hair in style,8/31/2015
4,US,107098,R35BHQJHXXJD59,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,Beauty,4,0,0,N,N,I think's great. The cord length is perfect,I just got this last week. I think's great. Th...,8/31/2015


In [110]:
specific_descriptor = ['wonderful', 'perfect', 'impress', 'excellent', 'awesome', 'beautiful', 'incredi', 'happy', 'attractive'
                       'awkward', 'disappoint', 'difficult', 'bad', 'wrong', 'hard']

In [111]:
import re
import nltk

from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet

#import nltk resources
resources = ["wordnet", "stopwords", "punkt", \
             "averaged_perceptron_tagger", "maxent_treebank_pos_tagger"]

for resource in resources:
    try:
        nltk.data.find("tokenizers/" + resource)
    except LookupError:
        nltk.download(resource)

#create Lemmatizer object
lemma = WordNetLemmatizer()

def lemmatize_word(tagged_token):
    """ Returns lemmatized word given its tag"""
    root = []
    for token in tagged_token:
        tag = token[1][0]
        word = token[0]
        if tag.startswith('J'):
            root.append(lemma.lemmatize(word, wordnet.ADJ))
        elif tag.startswith('V'):
            root.append(lemma.lemmatize(word, wordnet.VERB))
        elif tag.startswith('N'):
            root.append(lemma.lemmatize(word, wordnet.NOUN))
        elif tag.startswith('R'):
            root.append(lemma.lemmatize(word, wordnet.ADV))
        else:          
            root.append(word)
    return root

[nltk_data] Downloading package wordnet to /home/alphonse/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alphonse/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alphonse/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/alphonse/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


In [112]:
def lemmatize_doc(document):
    """ Tags words then returns sentence with lemmatized words"""
    lemmatized_list = []
    tokenized_sent = sent_tokenize(document)
    for sentence in tokenized_sent:
        no_punctuation = re.sub(r"[`'\",.!?()]", " ", sentence)
        tokenized_word = word_tokenize(no_punctuation)
        tagged_token = pos_tag(tokenized_word)
        lemmatized = lemmatize_word(tagged_token)
        lemmatized_list.extend(lemmatized)
    return " ".join(lemmatized_list)

In [113]:
from unicodedata import normalize



remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

In [114]:
from nltk.corpus import stopwords

stop_words = stopwords.words("english")

stop_words = [word.replace("\'", "") for word in stop_words]

print(f"sample stop words: {stop_words[:15]} \n")

remove_stop_words = lambda row: " ".join([token for token in row.split(" ") \
                                          if token not in stop_words])

sample stop words: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours'] 



In [115]:

pattern = r"\&\#[0-9]+\;"

df["preprocessed"] = df["review_body"].str.replace(pat=pattern, repl="", regex=True)

df["preprocessed"] = df["preprocessed"].apply(lambda row: lemmatize_doc(row))
df["preprocessed"] = df["preprocessed"].apply(remove_accent)

pattern = r"[^\w\s]"

df["preprocessed"] = df["preprocessed"].str.replace(pat=pattern, repl=" ", regex=True)

df["preprocessed"] = df["preprocessed"].str.lower()

df["preprocessed"] = df["preprocessed"].apply(remove_stop_words)

pattern = r"[\s]+"
df["preprocessed"] = df["preprocessed"].str.replace(pat=pattern, repl=" ", regex=True)

df.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,preprocessed
0,US,34678741,R9T1FE2ZX2X04,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,Beauty,5,0,0,N,Y,Works great,Works great!,8/31/2015,works great
1,US,11599505,RE36JAD5V53PO,B0009XH6V4,670161917,andis micro turbo hair dryer,Beauty,4,0,0,N,Y,I love travel blow dryers because they are eas...,"This dries my hair faster that bigger, more po...",8/31/2015,dry hair faster big powerful model love travel...
2,US,2282190,RIDHM8B7SCCV3,B0007NZPY6,16483457,conair pro hair dryer,Beauty,5,0,1,N,Y,Five Stars,Love this dryer!,8/31/2015,love dryer
3,US,43669858,R14QGWPCHU9LSE,B00BB8ZIW0,253917972,remington silk ceramic professional hair dryer,Beauty,5,0,0,N,Y,Five Stars,styling hair in style,8/31/2015,style hair style
4,US,107098,R35BHQJHXXJD59,B003V264WW,732252283,remington ac2015 t|studio salon collection pea...,Beauty,4,0,0,N,N,I think's great. The cord length is perfect,I just got this last week. I think's great. Th...,8/31/2015,get last week think great cord length perfect


In [116]:
print(len(df))

11470


In [117]:
sd_count = []
for i in range(len(specific_descriptor)):
    sd_count.append(list())
    for star in range(1, 6):
        star_count = []
        df_star = df[df['star_rating'] == star]
        corpora = df_star["preprocessed"].values
        
        tokenized = [corpus.split(" ") for corpus in corpora]
        count = 0
        for review in tokenized:
            for word in review:
                if specific_descriptor[i] in word:
                    count = count + 1
                    continue
                
        sd_count[i].append([star, count, len(df_star), count / len(df_star)])

In [118]:
sd_count

[[[1, 1, 1032, 0.0009689922480620155],
  [2, 8, 639, 0.012519561815336464],
  [3, 4, 999, 0.004004004004004004],
  [4, 19, 2096, 0.009064885496183206],
  [5, 125, 6704, 0.01864558472553699]],
 [[1, 15, 1032, 0.014534883720930232],
  [2, 17, 639, 0.026604068857589983],
  [3, 25, 999, 0.025025025025025027],
  [4, 114, 2096, 0.05438931297709924],
  [5, 583, 6704, 0.08696300715990453]],
 [[1, 11, 1032, 0.01065891472868217],
  [2, 12, 639, 0.018779342723004695],
  [3, 26, 999, 0.026026026026026026],
  [4, 29, 2096, 0.01383587786259542],
  [5, 101, 6704, 0.01506563245823389]],
 [[1, 7, 1032, 0.006782945736434108],
  [2, 2, 639, 0.003129890453834116],
  [3, 5, 999, 0.005005005005005005],
  [4, 49, 2096, 0.023377862595419848],
  [5, 292, 6704, 0.04355608591885442]],
 [[1, 4, 1032, 0.003875968992248062],
  [2, 1, 639, 0.001564945226917058],
  [3, 10, 999, 0.01001001001001001],
  [4, 23, 2096, 0.01097328244274809],
  [5, 170, 6704, 0.02535799522673031]],
 [[1, 4, 1032, 0.003875968992248062],
  [

In [164]:
dp_star = pd.DataFrame()

i = 10

dp_star['star'] = [x[0] for x in sd_count[i]]

dp_star['count'] = [x[1] for x in sd_count[i]]

dp_star['prop'] = [x[3] for x in sd_count[i]]

In [165]:
dp_star.head()


,star,count,prop
0,1,20,0.019380
1,2,23,0.035994
2,3,26,0.026026
3,4,39,0.018607
4,5,51,0.007607


In [166]:
dp_star.corr()

,star,count,prop
star,1.000000,0.952353,-0.619595
count,0.952353,1.000000,-0.769495
prop,-0.619595,-0.769495,1.000000
